In [31]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import seaborn as sns
import os

# Create output directory for saving visualizations
output_dir = "quantum_gate_visualizations"
os.makedirs(output_dir, exist_ok=True)

# Set random seed for reproducibility
np.random.seed(42)

In [32]:
# Define quantum gates as 2x2 unitary matrices
def hadamard_gate():
    """Hadamard gate: Creates superposition"""
    return (1/np.sqrt(2)) * np.array([[1, 1], 
                                       [1, -1]], dtype=complex)

def phase_gate():
    """Phase gate (S gate): Adds π/2 phase"""
    return np.array([[1, 0], 
                     [0, 1j]], dtype=complex)

def t_gate():
    """T gate: Adds π/4 phase"""
    return np.array([[1, 0], 
                     [0, np.exp(1j * np.pi / 4)]], dtype=complex)

In [33]:
def apply_gates(state, gates):
    """Apply a sequence of gates to a quantum state"""
    result = state.copy()
    for gate in gates:
        result = gate @ result
    return result

def state_to_bloch(state):
    """Convert quantum state to Bloch sphere coordinates"""
    # Normalize the state
    state = state / np.linalg.norm(state)
    
    # Extract amplitudes
    alpha = state[0]
    beta = state[1]
    
    # Calculate Bloch coordinates
    x = 2 * np.real(np.conj(alpha) * beta)
    y = 2 * np.imag(np.conj(alpha) * beta)
    z = np.abs(alpha)**2 - np.abs(beta)**2
    
    return x, y, z

In [34]:
def visualize_bloch_sphere(states_dict, title, filename):
    """Visualize quantum states on the Bloch sphere"""
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Draw the Bloch sphere
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x_sphere = np.outer(np.cos(u), np.sin(v))
    y_sphere = np.outer(np.sin(u), np.sin(v))
    z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x_sphere, y_sphere, z_sphere, alpha=0.1, color='cyan')
    
    # Draw axes
    ax.plot([-1, 1], [0, 0], [0, 0], 'k-', linewidth=0.5)
    ax.plot([0, 0], [-1, 1], [0, 0], 'k-', linewidth=0.5)
    ax.plot([0, 0], [0, 0], [-1, 1], 'k-', linewidth=0.5)
    
    # Label axes
    ax.text(1.2, 0, 0, 'X', fontsize=12, fontweight='bold')
    ax.text(0, 1.2, 0, 'Y', fontsize=12, fontweight='bold')
    ax.text(0, 0, 1.2, '|0⟩', fontsize=12, fontweight='bold')
    ax.text(0, 0, -1.2, '|1⟩', fontsize=12, fontweight='bold')
    
    # Plot states
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
    for idx, (label, state) in enumerate(states_dict.items()):
        x, y, z = state_to_bloch(state)
        ax.scatter([x], [y], [z], color=colors[idx % len(colors)], 
                  s=200, marker='o', edgecolors='black', linewidths=2, label=label)
        ax.plot([0, x], [0, y], [0, z], color=colors[idx % len(colors)], 
               linewidth=2, linestyle='--', alpha=0.7)
    
    ax.set_xlim([-1.2, 1.2])
    ax.set_ylim([-1.2, 1.2])
    ax.set_zlim([-1.2, 1.2])
    ax.set_box_aspect([1,1,1])
    ax.legend(loc='upper right', fontsize=10)
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    
    # Save the figure
    fig.savefig(f"{output_dir}/{filename}.png", dpi=300, bbox_inches='tight')
    print(f"✓ Saved Bloch sphere visualization: {output_dir}/{filename}.png")
    
    return fig

In [35]:
def compare_gate_orders():
    """Compare different orderings of H, P, and T gates"""
    
    # Define gates
    H = hadamard_gate()
    P = phase_gate()
    T = t_gate()
    
    # Initial state |0⟩
    initial_state = np.array([1, 0], dtype=complex)
    
    # Different gate orderings
    orderings = {
        'H → P → T': [H, P, T],
        'H → T → P': [H, T, P],
        'P → H → T': [P, H, T],
        'P → T → H': [P, T, H],
        'T → H → P': [T, H, P],
        'T → P → H': [T, P, H]
    }
    
    # Apply gates and store results
    results = {}
    print("="*80)
    print("QUANTUM GATE ORDER NON-COMMUTATIVITY DEMONSTRATION")
    print("="*80)
    print(f"\nInitial State: |0⟩ = {initial_state}")
    print("\nGates:")
    print(f"  H (Hadamard): Creates superposition")
    print(f"  P (Phase/S):  Adds π/2 phase to |1⟩")
    print(f"  T (T gate):   Adds π/4 phase to |1⟩")
    print("\n" + "="*80)
    
    for order_name, gates in orderings.items():
        final_state = apply_gates(initial_state, gates)
        results[order_name] = final_state
        
        print(f"\n{order_name}:")
        print(f"  Final state: [{final_state[0]:.4f}, {final_state[1]:.4f}]")
        print(f"  |α|² = {np.abs(final_state[0])**2:.4f}, |β|² = {np.abs(final_state[1])**2:.4f}")
        
        # Calculate the combined unitary matrix
        combined = np.eye(2, dtype=complex)
        for gate in gates:
            combined = gate @ combined
        print(f"  Combined Unitary Matrix:")
        for row in combined:
            print(f"    [{row[0]:>12.4f}, {row[1]:>12.4f}]")
    
    return results, orderings